In [1]:
# import libraries
import FinanceDataReader as fdr
import numpy as np
from datetime import datetime, timedelta
import math

In [2]:
# 데이터 수집 기간 및 종목코드 설정
start_date = '20140106'
end_date = '20220527'
code = '326030' # 20년 7월 2일 상장 종목으로 테스트

In [3]:
# 시작일이 주말인 경우: 예외처리
start_datetime = datetime.strptime(start_date, '%Y%m%d')
if (start_datetime.weekday() == 5): # 시작: 토요일
    start_datetime += timedelta(days=2)
    start_date = start_datetime.strftime('%Y%m%d')
elif (start_datetime.weekday() == 6): # 시작: 일요일
    start_datetime += timedelta(days=1)
    start_date = start_datetime.strftime('%Y%m%d')

In [4]:
# 종목코드에 대한 종가 데이터 수집
data = fdr.DataReader(code, start_date, end_date)
data.drop(['Open', 'High', 'Low', 'Change', 'Volume'], axis=1, inplace=True)

# 5/20/60일 이동평균 계산
ma5 = data['Close'].rolling(window=5).mean()
data['ma5'] = ma5
ma20 = data['Close'].rolling(window=20).mean()
data['ma20'] = ma20
ma60 = data['Close'].rolling(window=60).mean()
data['ma60'] = ma60

# NaN 데이터 제거: ma5는 Close로, ma20은 ma5로, ma60은 ma20으로 대체
for idx in data.index:
    if (math.isnan(data.loc[idx, 'ma5'])):
        data.loc[idx, 'ma5'] = data.loc[idx, 'ma20'] = data.loc[idx, 'ma60'] = data.loc[idx, 'Close']
    if (math.isnan(data.loc[idx, 'ma20'])):
        data.loc[idx, 'ma20'] = data.loc[idx, 'ma60'] = data.loc[idx, 'ma5']
    if (math.isnan(data.loc[idx, 'ma60'])):
        data.loc[idx, 'ma60'] = data.loc[idx, 'ma20']

# 이동평균 변화율(Change) 계산
data['ma5_Change'] = np.ceil((data['ma5']/data['ma5'].shift(1) - 1) * 100 * 10000)/10000
data['ma20_Change'] = np.ceil((data['ma20']/data['ma20'].shift(1) - 1) * 100 * 10000)/10000
data['ma60_Change'] = np.ceil((data['ma60']/data['ma60'].shift(1) - 1) * 100 * 10000)/10000
data.fillna(method='bfill', inplace=True)
        
# 이동평균 데이터타입 변환 (float to int)
data['ma5'] = data['ma5'].astype(int)
data['ma20'] = data['ma20'].astype(int)
data['ma60'] = data['ma60'].astype(int)

print(data)

             Close     ma5    ma20    ma60  ma5_Change  ma20_Change  \
Date                                                                  
2020-07-02  127000  127000  127000  127000     29.9213      29.9213   
2020-07-03  165000  165000  165000  165000     29.9213      29.9213   
2020-07-06  214500  214500  214500  214500     30.0001      30.0001   
2020-07-07  216500  216500  216500  216500      0.9325       0.9325   
2020-07-08  217000  188000  188000  188000    -13.1639     -13.1639   
...            ...     ...     ...     ...         ...          ...   
2022-05-23   86600   87100   88195   88066     -0.4798      -0.0113   
2022-05-24   85500   86500   88135   88193     -0.6888      -0.0680   
2022-05-25   87400   86360   88065   88285     -0.1618      -0.0794   
2022-05-26   87700   86700   87970   88336      0.3938      -0.1078   
2022-05-27   87700   86980   87845   88375      0.3230      -0.1420   

            ma60_Change  
Date                     
2020-07-02      29.9213 